In [1]:
%pip install statsforecast
%pip install statsmodels
%pip install pmdarima
%pip install pandas
%pip install plotly
%pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# YEARLY SUNSPOT DATA

In [2]:
from statsforecast.models import ARIMA
from pmdarima.arima import auto_arima
import plotly.graph_objs as go
import statsmodels.api as sm
from arima import Model

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [3]:
H = 45
DS = "YEAR"
Y = "SUNACTIVITY"

In [4]:
df = sm.datasets.sunspots.load_pandas().data
y_train = df[Y].values[:-H]

# Search hyper-parameters

In [5]:
# search = auto_arima(
#     y=y_train,
#     start_p=0,
#     max_p=5,
#     d=0,
#     max_d=1,
#     start_q=0,
#     max_q=5,
#     start_P=0,
#     max_P=2,
#     start_Q=0,
#     max_Q=2,
#     D=1,
#     seasonal=True,
#     m=43,
#     stepwise=True,
#     maxiter=50,
#     information_criterion="aic",
#     suppress_warnings=True,
#     trace=True
# )

In [14]:
# params = search.get_params()
# order = params["order"]
order = (2, 0, 1)
# order = (3, 0, 0)
# seasonal_order = params["seasonal_order"]
seasonal_order = (2, 1, 0, 43)
# params

# Train arima models

### My Arima

In [15]:
%%timeit
m1 = Model.sarima(order=order, seasonal_order=seasonal_order)
m1.fit(y=y_train)
preds1 = m1.predict(h=H, x=None)

410 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### StatsForecast

In [16]:
%%timeit
m2 = ARIMA(order=order, seasonal_order=seasonal_order[: -1], season_length=seasonal_order[-1])
preds2 = m2.forecast(y=y_train, h=H)["mean"]

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt



1min 4s ± 1.87 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\arima.py:1158: RuntimeWarning:

invalid value encountered in sqrt



### Statsmodels

In [9]:
m3 = (sm.tsa.statespace
      .SARIMAX(endog=y_train, order=order, seasonal_order=seasonal_order)
      .fit()
)
preds3 = m3.predict(start=y_train.size, end=y_train.size + H - 1)

# Plot

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df[DS], y=df[Y], mode='lines', name=Y))
test_dates = df[DS].values[-H:]
fig.add_trace(go.Scatter(x=test_dates, y=preds1, mode='lines', name="MyArima"))
fig.add_trace(go.Scatter(x=test_dates, y=preds2, mode='lines', name="StatsForecast"))
fig.add_trace(go.Scatter(x=test_dates, y=preds3, mode='lines', name="Statsmodels"))
fig.update_layout(showlegend=True)
fig.show()